# Exporting a cell in the neuroml format and running it

To run this notebook, it is necessary to have bluepyopt, pyneuroml and libNeuroML installed. This can be achieved with the following pip install:

In [ ]:
pip install bluepyopt[neuroml]

Note that the bluepyopt neuroml module cannot yet handle:
- non uniform parameter
- axon replacement
- stochasticity

## Exporting a cell to neuroml

In [11]:
import os
import sys

from pyneuroml import pynml
from bluepyopt.neuroml import cell
from bluepyopt.neuroml import simulation

For simplicity, in this notebook we will use the cell from bluepyopt's l5pc example:

In [13]:
L5PC_PATH = os.path.abspath("../l5pc")
sys.path.insert(0, L5PC_PATH)

import l5pc_model

l5pc_cell = l5pc_model.create()

We have to disable replace_axon, since it is not supported by bluepyopt's neuroml module yet:

In [14]:
l5pc_cell.morphology.do_replace_axon = False

We have to define the cell's optimised parameters:

In [15]:
release_params = {
    "gNaTs2_tbar_NaTs2_t.apical": 0.026145,
    "gSKv3_1bar_SKv3_1.apical": 0.004226,
    "gImbar_Im.apical": 0.000143,
    "gNaTa_tbar_NaTa_t.axonal": 3.137968,
    "gK_Tstbar_K_Tst.axonal": 0.089259,
    "gamma_CaDynamics_E2.axonal": 0.002910,
    "gNap_Et2bar_Nap_Et2.axonal": 0.006827,
    "gSK_E2bar_SK_E2.axonal": 0.007104,
    "gCa_HVAbar_Ca_HVA.axonal": 0.000990,
    "gK_Pstbar_K_Pst.axonal": 0.973538,
    "gSKv3_1bar_SKv3_1.axonal": 1.021945,
    "decay_CaDynamics_E2.axonal": 287.198731,
    "gCa_LVAstbar_Ca_LVAst.axonal": 0.008752,
    "gamma_CaDynamics_E2.somatic": 0.000609,
    "gSKv3_1bar_SKv3_1.somatic": 0.303472,
    "gSK_E2bar_SK_E2.somatic": 0.008407,
    "gCa_HVAbar_Ca_HVA.somatic": 0.000994,
    "gNaTs2_tbar_NaTs2_t.somatic": 0.983955,
    "decay_CaDynamics_E2.somatic": 210.485284,
    "gCa_LVAstbar_Ca_LVAst.somatic": 0.000333,
}

And we have to compile the mechanisms:

In [ ]:
os.system("nrnivmodl ../l5pc/mechanisms/")

Finally, we can create the neuroml cell. Using `skip_channels_copy=False` will copy the neuroml mechanisms from `bluepyopt/neuroml/NeuroML2_mechanisms/` to a folder named `channels`.

This creates:
- a neuroml cell file named after the bluepyopt cell's name: Here, `l5pc_0_0.cell.nml`.
- a neuroml network file containing the neuroml cell, named after the bluepyopt cell's name. Here, `l5pc.net.nml`.

Skip this step if you want to use custom mechanisms not present in `bluepyopt/neuroml/NeuroML2_mechanisms/`.

In [ ]:
cell.create_neuroml_cell(
    l5pc_cell, release_params, skip_channels_copy=False,
)

If you want to create a neuroml cell with custom mechanisms that are not present in `bluepyopt/neuroml/NeuroML2_mechanisms/`, you will have to:
- copy your custom mechanisms in the `.nml` format in the `./channels` directory
- give as argument `custom_channel_ion`, a dict mapping channel name to ion name, e.g. `custom_channel_ion = {"NaCustom": "na"}`
- if one of the ion in `custom_channel_ion` is not in pre-registered ions (na, k, hcn, ca, pas), you'll also have to give as argument `custom_ion_erevs`, a dict mapping ions to their reversal potential

Below is an example of how to use the `create_neuroml_cell` function with custom mechanisms.

However, be aware that `create_neuroml_cell` might not be able to deal with any given custom mechanism, and some might break it.

In [ ]:
custom_channel_ion = {"Cl_custom_mech_name": "cl"}
custom_ion_erevs = {"cl": "-60.0 mV"}

cell.create_neuroml_cell(
    l5pc_cell, release_params, skip_channels_copy=False, custom_channel_ion=custom_channel_ion, custom_ion_erevs=custom_ion_erevs,
)

## Creating a LEMS simulation able to run the neuroml cell

First, we have to input the name of the neuroml network created with the cell at the previous step.

In [18]:
network_filename = f"{l5pc_cell.name}.net.nml"

We have to get the protocols. Here, we can get the ones from the bluepyopt l5pc example.

In [ ]:
import l5pc_evaluator

protocols = l5pc_evaluator.define_protocols()
protocol_name = "Step3"
bpo_test_protocol = protocols[protocol_name]

We also have to define a timestep.

In [20]:
dt = 0.025

Finally, we have to define the name of the LEMS simulation file.

In [21]:
lems_filename = f"LEMS_{l5pc_cell.name}.xml"

We can  now create the LEMS simulation file.

In [ ]:
simulation.create_neuroml_simulation(
    network_filename, bpo_test_protocol, dt, l5pc_cell.name, lems_filename
)

## Run the simulation

First, we have to remove the compiled mechanisms for the LEMS simulation to run without issues.

In [ ]:
os.system("rm -rf x86_64/")

Before running the LEMS simulation, you might have to set the env variable NEURON_HOME in your jupyter kernel. Change the following jupyter cell accordingly to set your NEURON_HOME variable. It should link to the parent directory of the nrniv executable, up to but not including bin.

In [ ]:
%env NEURON_HOME=path/to/neuron/home

Since the default jNeuroML simulator can only simulate single compartment cells, we will run the simulation with the NEURON simulator.

In [ ]:
pynml.run_lems_with_jneuroml_neuron(
    lems_filename, nogui=True, plot=False
)

This will output a trace in the following file: `l5pc.Pop_l5pc_0_0.v.dat`. Note that the data are recorded in volts and seconds.